# English to Bahasa Indonesian Seq-2-Seq Translation

In [1]:
# IPython candies...
from IPython.display import Image
from IPython.core.display import HTML 
# Imports we need.
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import pandas as pd

use_cuda = torch.cuda.is_available()

## Data Munging

In [2]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from nltk import word_tokenize

#fp = open('./corpus/bbc-468.en', 'r')
fp = open('./corpus/SMERU-26870.en', 'r')
eng_text = fp.read()
eng_text = eng_text.splitlines()
fp.close()

#fp2 = open('./corpus/bbc-468.id', 'r')
fp2 = open('./corpus/SMERU-26870.id', 'r')
id_text = fp2.read()
id_text = id_text.splitlines()
fp2.close()

In [3]:
df_eng_text = pd.DataFrame(eng_text)
df_eng_text = df_eng_text.rename(columns={0:'English'})

df_id_text = pd.DataFrame(id_text)
df_id_text = df_id_text.rename(columns={0:'Indonesian'})


In [4]:
df_eng_text['English'] = df_eng_text['English'].apply(lambda x : x.lstrip())
df_id_text['Indonesian'] = df_id_text['Indonesian'].apply(lambda x : x.lstrip())

In [5]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from nltk import word_tokenize

df = pd.concat([df_eng_text, df_id_text], axis=1)
df.head()

# corpus_df = pd.read_csv("./corpus/trimmed_combined_no_duplicate.csv")
# corpus_df = corpus_df.drop(columns=["English_num_words", "Indo_num_words"])
# corpus_df.head()

,English,Indonesian
0,ACKNOWLEDGEMENTS This report of Access and Equ...,UCAPAN TERIMA KASIH Laporan mengenai Akses ter...
1,We would like to express our genuine appreciat...,Kami ingin menyampaikan apresiasi yang sebesar...
2,We are truly grateful to the Family Court of A...,Kami sangat berterima kasih kepada Family Cour...
3,We would also like to express our sincere grat...,Kami juga ingin menyampaikan rasa terima kasih...
4,We especially appreciate the support and accep...,Kami berterima kasih secara khusus atas dukung...


In [6]:
# corpus_df['English'] = corpus_df['English'].apply(lambda x : x.lstrip())
# corpus_df['Indonesian'] = corpus_df['Indonesian'].apply(lambda x : x.lstrip())

In [7]:
# Use a unique string to indicate START and END of a sentence.
# Assign a unique index to them.
START, START_IDX = '<s>',  0
END, END_IDX = '</s>', 1

# We use this idiom to tokenize our sentences in the dataframe column:
# >>> DataFrame['column'].apply(str.lower).apply(word_tokenize)

# Also we added the START and the END symbol to the sentences. 
english_sents = [START] + df_eng_text['English'].apply(str.lower).apply(word_tokenize) + [END]
indo_sents = [START] + df_id_text['Indonesian'].apply(str.lower).apply(word_tokenize) + [END]

# We're sort of getting into the data into the shape we want. 
# But now it's still too humanly readable and redundant.
## Cut-away: Computers like it to be simpler, more concise. -_-|||
print('First Singlish sentence:', english_sents[0])
print('First English sentence:', indo_sents[0])

First Singlish sentence: ['<s>', 'acknowledgements', 'this', 'report', 'of', 'access', 'and', 'equity', 'survey', 'in', 'family', 'law', 'and', 'civil', 'status', 'issues', 'for', 'the', 'courts', 'in', 'indonesia', 'could', 'only', 'be', 'finished', 'with', 'the', 'support', 'and', 'cooperation', 'of', 'a', 'number', 'of', 'people', '</s>']
First English sentence: ['<s>', 'ucapan', 'terima', 'kasih', 'laporan', 'mengenai', 'akses', 'terhadap', 'keadilan', 'pemberdayaan', 'perempuan', 'kepala', 'keluarga', 'di', 'indonesia', 'ini', 'hanya', 'dapat', 'terselesaikan', 'berkat', 'dukungan', 'dan', 'kerja', 'sama', 'dari', 'seluruh', 'pihak', 'yang', 'terlibat', '</s>']


In [8]:
english_vocab = Dictionary([['<s>'], ['</s>']])
english_vocab.add_documents(english_sents)

indo_vocab = Dictionary([['<s>'], ['</s>']])
indo_vocab.add_documents(indo_sents)

# First ten words in the vocabulary.
print('First 10 Indonesian words in Dictionary:\n', sorted(indo_vocab.items())[:10])
print()
print('First 10 English words in Dictionary:\n', sorted(english_vocab.items())[:10])

First 10 Indonesian words in Dictionary:
 [(0, '<s>'), (1, '</s>'), (2, 'akses'), (3, 'berkat'), (4, 'dan'), (5, 'dapat'), (6, 'dari'), (7, 'di'), (8, 'dukungan'), (9, 'hanya')]

First 10 English words in Dictionary:
 [(0, '<s>'), (1, '</s>'), (2, 'a'), (3, 'access'), (4, 'acknowledgements'), (5, 'and'), (6, 'be'), (7, 'civil'), (8, 'cooperation'), (9, 'could')]


In [9]:
import pickle
# Lets save our dictionaries.
with open('./vocabs/indo_vocab.Dictionary.pkl', 'wb') as fout:
    pickle.dump(indo_vocab, fout)
    
with open('./vocabs/english_vocab.Dictionary.pkl', 'wb') as fout:
    pickle.dump(english_vocab, fout)

In [10]:
# Vectorizes a sentence with a given vocab
def vectorize_sent(sent, vocab):
    return vocab.doc2idx([START] + word_tokenize(sent.lower()) + [END])

# Creates a PyTorch variable from a sentence against a given vocab
def variable_from_sent(sent, vocab):
    vsent = vectorize_sent(sent, vocab)
    #print(vsent)
    result = Variable(torch.LongTensor(vsent).view(-1, 1))
    #print(result)
    return result.cuda() if use_cuda else result

# Test
new_kopi = "French Muslims fined for face veils"
variable_from_sent(new_kopi, english_vocab)

tensor([[   0],
        [  -1],
        [1970],
        [7057],
        [  14],
        [3521],
        [  -1],
        [   1]], device='cuda:0')

In [11]:
# Prepare the whole training corpus.
indo_tensors = df_id_text['Indonesian'].apply(lambda s: variable_from_sent(s, indo_vocab))
print(df_id_text.iloc[0]['Indonesian'])
print(indo_tensors[0])
english_tensors = df_eng_text['English'].apply(lambda s: variable_from_sent(s, english_vocab))
print(df_eng_text.iloc[0]['English'])
print(english_tensors[0])
# Now, each item in `sent_pairs` is our data point. 
sent_pairs = list(zip(english_tensors, indo_tensors))

UCAPAN TERIMA KASIH Laporan mengenai Akses terhadap Keadilan Pemberdayaan Perempuan Kepala Keluarga di Indonesia ini hanya dapat terselesaikan berkat dukungan dan kerja sama dari seluruh pihak yang terlibat
tensor([[ 0],
        [28],
        [25],
        [12],
        [17],
        [18],
        [ 2],
        [24],
        [13],
        [19],
        [20],
        [15],
        [14],
        [ 7],
        [10],
        [11],
        [ 9],
        [ 5],
        [27],
        [ 3],
        [ 8],
        [ 4],
        [16],
        [22],
        [ 6],
        [23],
        [21],
        [29],
        [26],
        [ 1]], device='cuda:0')
ACKNOWLEDGEMENTS This report of Access and Equity Survey in Family Law and Civil Status Issues for the Courts in Indonesia could only be finished with the support and cooperation of a number of people
tensor([[ 0],
        [ 4],
        [28],
        [23],
        [20],
        [ 3],
        [ 5],
        [11],
        [26],
        [15],
        [12],


# Seq2Seq Model

## Encoder Definition

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # Set the no. of nodes for the hidden layer.
        self.hidden_size = hidden_size
        # Initialize the embedding layer with the 
        # - size of input (i.e. no. of words in input vocab)
        # - no. of hidden nodes in the embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)
        # Initialize the GRU with the 
        # - size of the hidden layer from the previous state
        # - size of the hidden layer from the current state
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Feed the input into the embedding layer.
        embedded = self.embedding(input).view(1, 1, -1) # Question: why (1, 1, -1)
        # Feed the embedded layer with the hidden layer to the GRU.
        # Update the output and hidden layer.
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initialize_hidden_states(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result.cuda() if use_cuda else result


# Decoder Definition

In [13]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        # Set the no. of nodes for the hidden layer.
        self.hidden_size = hidden_size
        # Initialize the embedding layer with the 
        # - size of output (i.e. no. of words in output vocab)
        # - no. of hidden nodes in the embedding layer
        self.embedding = nn.Embedding(output_size, hidden_size)
        # Initialize the GRU with the 
        # - size of the hidden layer from the previous state
        # - size of the hidden layer from the current state
        self.gru = nn.GRU(hidden_size, hidden_size)
        # Set the output layer to output a specific symbol 
        # from the output vocabulary
        self.softmax = nn.LogSoftmax(dim=1)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        # Feed the input into the embedding layer.
        output = self.embedding(input).view(1, 1, -1)
        # Transform the embedded output with a relu function. 
        output = F.relu(output)
        # Feed the embedded layer with the hidden layer to the GRU.
        # Update the output and hidden layer.
        output, hidden = self.gru(output, hidden)
        # Take the updated output and find the most appropriate
        # output symbol. 
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initialize_hidden_states(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        return result.cuda() if use_cuda else result
        

In [14]:
hidden_size = 100
learning_rate=0.01
batch_size = 2
epochs = 30 # Since we are taking batch_size=2 and epochs=30, we only look at 60 data points.
criterion = nn.NLLLoss()
MAX_LENGTH=80

# Initialize the network for encoder and decoder.
input_vocab, output_vocab = english_vocab, indo_vocab
encoder = EncoderRNN(len(input_vocab), hidden_size)
decoder = DecoderRNN(hidden_size, len(output_vocab))
if use_cuda:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

# Initialize the optimizer for encoder and decoder.
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

# If batchsize == 1, choose 1 data points per batch:
##training_data = [[random.choice(sent_pairs)] for i in range(epochs)]

# If batch_size > 1, use random.sample() instead of random.choice:
training_data = [random.sample(sent_pairs, batch_size) for i in range(epochs)]

## Checking the encoder

In [15]:
#############################################
# 2.3.2. Loop through the batches.
#############################################
# Start the training.
for data_batch in training_data:
    # (Re-)Initialize the optimizers, clear all gradients after every batch.
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset the loss for every batch.
    loss = 0
    for input_variable, target_variable in data_batch:
        # Initialize the hidden_states for the encoder.
        encoder_hidden = encoder.initialize_hidden_states()
        # Initialize the length of the PyTorch variables.
        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]
        encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

In [16]:
print("Input variable first: {0}".format(data_batch[0][0]))
print("Target variable first: {0}".format(data_batch[0][1]))
print(input_length)

Input variable first: tensor([[   0],
        [  28],
        [ 150],
        [  31],
        [1098],
        [  31],
        [1315],
        [4239],
        [  14],
        [  27],
        [1724],
        [   1]], device='cuda:0')
Target variable first: tensor([[   0],
        [ 254],
        [ 175],
        [1044],
        [ 726],
        [4639],
        [ 149],
        [ 353],
        [   1]], device='cuda:0')
13


In [17]:
#############################################
# 2.3.2. Loop through the batches.
#############################################
# Start the training.
for data_batch in training_data:
    # (Re-)Initialize the optimizers, clear all gradients after every batch.
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset the loss for every batch.
    loss = 0
    for input_variable, target_variable in data_batch:
        # Initialize the hidden_states for the encoder.
        encoder_hidden = encoder.initialize_hidden_states()
        # Initialize the length of the PyTorch variables.
        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]
        encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
        #############################################
        # 2.3.3.  Iterating through each word in the encoder.
        #############################################
        # Iterating through each word in the input.
        for ei in range(input_length):
            # We move forward through each state.
            encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
            # And we save the encoder outputs. 
            # Note: We're retrieving [0][0] cos remember the weird .view(1,1,-1) for the GRU.
            encoder_outputs[ei] = encoder_output[0][0] 

In [18]:
# The encoded output for the last sentence in out training_data"

# The encoder has 68 unique words
print(encoder, '\n')
print(english_vocab)
print('\n########\n')

EncoderRNN(
  (embedding): Embedding(16962, 100)
  (gru): GRU(100, 100)
) 

Dictionary(16962 unique tokens: ['<s>', '</s>', 'a', 'access', 'acknowledgements']...)

########



In [19]:
input_length


13

In [20]:
# The last input sentence, in PyTorch Tensor data structure.
print(data_batch[-1][0]) 
print('########\n')


tensor([[   0],
        [ 319],
        [  68],
        [ 987],
        [1190],
        [1201],
        [  67],
        [1188],
        [1196],
        [7677],
        [  68],
        [  31],
        [   1]], device='cuda:0')
########



In [21]:

# The last input sentence as list(int)
print(list(map(int, data_batch[-1][0])), '\n')
print('########\n')

# The last input sentence as list(int)
print(' '.join([english_vocab[i] for i in map(int, data_batch[-1][0])]))
print('\n########\n')


[0, 319, 68, 987, 1190, 1201, 67, 1188, 1196, 7677, 68, 31, 1] 

########

<s> e ) direct cash transfer ( bantuan langsung tunai—blt ) , </s>

########



In [22]:
print(encoder_outputs)
# The last hidden state of the last input sentence. 
# Note: For vanilla RNN (Elman Net), the last hidden state of the encoder
#       is the start state of the decoder's hidden state.
print(encoder_hidden)

tensor([[-0.3013,  0.2890,  0.0450,  ..., -0.0463, -0.3196,  0.1635],
        [-0.1497,  0.2339,  0.1516,  ...,  0.3822, -0.2726, -0.2556],
        [ 0.1998,  0.2763, -0.1425,  ...,  0.3614,  0.6233, -0.2843],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<CopySlices>)
tensor([[[ 0.2247,  0.2115, -0.2333, -0.1634, -0.2026, -0.1118,  0.2499,
           0.2703,  0.1442,  0.1590, -0.3202, -0.3676, -0.6249,  0.4999,
          -0.0024, -0.3273,  0.2060, -0.1906, -0.4942,  0.4265,  0.2896,
          -0.0341, -0.6040,  0.4384,  0.1585,  0.4642, -0.0460, -0.3187,
           0.6347, -0.3133,  0.1301, -0.3813,  0.2708,  0.0977,  0.0499,
           0.3580, -0.2335,  0.1643, -0.1968, -0.0036,  0.1587, -0.2451,
           0.3282, -0.6705,  0.1914,  0.2036, -0.0670, -0.4116,  0.6279,
         

## Checking the decoder 

In [23]:
#############################################
# 2.3.2. Loop through the batches.
#############################################
# Start the training.
for data_batch in training_data:
    # (Re-)Initialize the optimizers, clear all gradients after every batch.
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset the loss for every batch.
    loss = 0
    for input_variable, target_variable in data_batch:
        # Initialize the hidden_states for the encoder.
        encoder_hidden = encoder.initialize_hidden_states()
        # Initialize the length of the PyTorch variables.
        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]
        encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
        
        #############################################
        # 2.3.3.  Iterating through each word in the encoder.
        #############################################
        # Iterating through each word in the input.
        for ei in range(input_length):
            # We move forward through each state.
            encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
            # And we save the encoder outputs. 
            # Note: We're retrieving [0][0] cos remember the weird .view(1,1,-1) -_-|||
            encoder_outputs[ei] = encoder_output[0][0] 
            
            #############################################
            # 2.3.4.  Iterating through each word in the decoder.
            #############################################
            # Initialize the variable input with the index of the START.
            decoder_input = Variable(torch.LongTensor([[START_IDX]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            # As the first state of the decoder, we take the last step of the encoder.
            decoder_hidden = encoder_hidden
            # Iterate through each state in the decoder.
            # Note: when we are training we know the length of the decoder.
            #       so we can use the trick to restrict the loop when decoding.
            for di in range(target_length):
                # We move forward through each state.
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # What are all these weird syntax, refer to 2.3.4.1
                topv, topi = decoder_output.data.topk(1) # topk gives k largest values along a certain dimension, A tuple of (values, indices) is returned,
                ni = topi[0][0]

                # Replace our decoder input for the next state with the
                # embedding of the decoded topi guess. 
                decoder_input = Variable(torch.LongTensor([[ni]]))
                decoder_input = decoder_input.cuda() if use_cuda else decoder_input
                
                # Update our loss for this batch.
                loss += criterion(decoder_output, target_variable[di])
                
                # If we see the </s> symbol, break the training.
                if ni == END_IDX:
                    break


In [24]:
# Cut-away: The decoded output for the last sentence in out training_data"

# The encoder has 117 unique words
print(decoder, '\n')
print(indo_vocab)
print('\n########\n')

# The last input sentence.
print(' '.join([english_vocab[i] for i in map(int, data_batch[-1][0])]))
# The last target sentence.
print(' '.join([indo_vocab[i] for i in map(int, data_batch[-1][1])]))

print('\n########\n')

DecoderRNN(
  (embedding): Embedding(16597, 100)
  (gru): GRU(100, 100)
  (softmax): LogSoftmax()
  (out): Linear(in_features=100, out_features=16597, bias=True)
) 

Dictionary(16597 unique tokens: ['<s>', '</s>', 'akses', 'berkat', 'dan']...)

########

<s> e ) direct cash transfer ( bantuan langsung tunai—blt ) , </s>
<s> e ) bantuan langsung tunai ( blt ) , </s>

########



## Actual Training Starts Here - Backpropagation Portion Added

In [25]:
#############################################
# 2.3.2. Loop through the batches.
#############################################
# Start the training.
for data_batch in training_data:
    # (Re-)Initialize the optimizers, clear all gradients after every batch.
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Reset the loss for every batch.
    loss = 0
    for input_variable, target_variable in data_batch:
        # Initialize the hidden_states for the encoder.
        encoder_hidden = encoder.initialize_hidden_states()
        # Initialize the length of the PyTorch variables.
        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]
        encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
        
        #############################################
        # 2.3.3.  Iterating through each word in the encoder.
        #############################################
        # Iterating through each word in the input.
        for ei in range(input_length):
            # We move forward through each state.
            encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
            # And we save the encoder outputs. 
            # Note: We're retrieving [0][0] cos remember the weird .view(1,1,-1) -_-|||
            encoder_outputs[ei] = encoder_output[0][0] 
            
            #############################################
            # 2.3.4.  Iterating through each word in the decoder.
            #############################################
            # Initialize the variable input with the index of the START.
            decoder_input = Variable(torch.LongTensor([[START_IDX]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            # As the first state of the decoder, we take the last step of the encoder.
            decoder_hidden = encoder_hidden
            # Iterate through each state in the decoder.
            # Note: when we are training we know the length of the decoder.
            #       so we can use the trick to restrict the loop when decoding.
            for di in range(target_length):
                # We move forward through each state.
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # What are alll these weird syntax, refer to 2.3.4.1
                topv, topi = decoder_output.data.topk(1)
                ni = topi[0][0]

                # Replace our decoder input for the next state with the
                # embedding of the decoded topi guess. 
                decoder_input = Variable(torch.LongTensor([[ni]]))
                decoder_input = decoder_input.cuda() if use_cuda else decoder_input
                
                # Update our loss for this batch.
                loss += criterion(decoder_output, target_variable[di])
                
                # If we see the </s> symbol, break the training.
                if ni == END_IDX:
                    break
    #####################################################
    # 2.3.5 Backpropagate the Loss and Optimizers Takes a Step.
    #####################################################
    loss.backward() # Backpropagate.
    encoder_optimizer.step()
    decoder_optimizer.step()


In [26]:
MAX_LENGTH

80

In [27]:
encoder_outputs.shape

torch.Size([80, 100])

In [28]:
def translator(encoder, decoder, input_variable, max_length=MAX_LENGTH):
    # The length of the input.
    input_length = input_variable.size()[0]
    # For each sentence, initilize the hidden states with zeros.
    encoder_hidden = encoder.initialize_hidden_states()
    # Initialize the encoder outputs. 
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
    # Iterate through the input words.
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
    # Initialize the decoder with the start symbol <s>.
    decoder_input = Variable(torch.LongTensor([[START_IDX]])) 
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    # Use the last encoder hidden state as the first decoder's hidden state.
    decoder_hidden = encoder_hidden
    # Keep a list of the decoded words.
    decoded_words = []
    
    # Iterate through the decoder states.
    for di in range(max_length):
        # Very similar to how the training works.
        decoder_output, decoder_hidden = decoder(decoder_input, 
                                                 decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == END_IDX:
            decoded_words.append(END_IDX)
            break
        else:
            decoded_words.append(ni.item())# changed from ni to ni.item()
        # Replace the new decoder input for the next state 
        # with the top guess of this state.
        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words

In [29]:
sent = 'caffeine protects the brain' # if words don't appear in corpus, it will be marked as -1
variable_from_sent(sent, english_vocab)

tensor([[ 0],
        [-1],
        [-1],
        [27],
        [-1],
        [ 1]], device='cuda:0')

In [30]:
MAX_LENGTH = 80
batches = 100000 # In this case, the PyTorch train_per_epoch() and train() code is using batch_size=1
hidden_size = 10

my_encoder = EncoderRNN(len(english_vocab), hidden_size)
my_decoder = DecoderRNN(hidden_size, len(indo_vocab))

if use_cuda:
    my_encoder = my_encoder.cuda()
    my_decoder = my_decoder.cuda()

In [31]:
output_words = translator(my_encoder, my_decoder, 
                          variable_from_sent(sent, english_vocab))
len(output_words)

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

In [ ]:
[indo_vocab[i] for i in output_words[1:30]]

In [ ]:
#########################################################
# Some Logging and Plotting Candies to Monitor Training
#########################################################

import time
import math

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    
    
#########################################################
# Training per epoch,
# Iterates across data points per epoch.
#########################################################
def train_one_epoch(input_variable, target_variable, encoder, decoder, 
                    encoder_optimizer, decoder_optimizer, criterion):
    """
    Function to put the variables, decoder and optimizers to train per epoch.
    """
    encoder_hidden = encoder.initialize_hidden_states()

    # (Re-)Initialize the optimizers, clear all gradients. 
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Initialize the length of the PyTorch variables.
    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0
    
    # Iterating through each word in the input.
    for ei in range(input_length):
        # We move forward through each state.
        encoder_output, encoder_hidden = encoder(input_variable[ei], encoder_hidden)
        # And we save the encoder outputs. 
        encoder_outputs[ei] = encoder_output[0][0]

    # Initialize the variable input with the index of the START.
    decoder_input = Variable(torch.LongTensor([[START_IDX]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    # As the first state of the decoder, we take the last step of the encoder.
    decoder_hidden = encoder_hidden
    
    # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        loss += criterion(decoder_output, target_variable[di])
        if ni == END_IDX:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    #return loss.data[0] / target_length
    return loss.item() / target_length

#########################################################
# Top-level function to start the training,
# iterates across epochs.
#########################################################

def train(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [random.choice(sent_pairs) for i in range(n_iters)]
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train_one_epoch(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    
    showPlot(plot_losses)

In [ ]:
MAX_LENGTH = 80
batches = 6000 # In this case, the PyTorch train_per_epoch() and train() code is using batch_size=1
hidden_size = 100

my_encoder = EncoderRNN(len(english_vocab), hidden_size)
my_decoder = DecoderRNN(hidden_size, len(indo_vocab))

if use_cuda:
    my_encoder = my_encoder.cuda()
    my_decoder = my_decoder.cuda()

train(my_encoder, my_decoder, batches, print_every=100)

In [32]:
# Here's a nice bleeding edge Python trick, (only works on Python3.6)
# F-strings for the win!!
# See https://www.python.org/dev/peps/pep-0498/
print(hidden_size, batches)
print(f'encoder_vanilla_{hidden_size}_{batches}.pkl')

10 100000
encoder_vanilla_10_100000.pkl


In [33]:
import pickle 

# In Python >= 3.6
with open(f'./models/encoder_vanilla_{hidden_size}_{batches}.pkl', 'wb') as fout:
    pickle.dump(my_encoder, fout)
with open(f'./models/decoder_vanilla_{hidden_size}_{batches}.pkl', 'wb') as fout:
    pickle.dump(my_decoder, fout)

# For Python < 3.6
with open('./models/encoder_vanilla_{}_{}.pkl'.format(hidden_size, batches), 'wb') as fout:
    pickle.dump(my_encoder, fout)
with open('./models/decoder_vanilla_{}_{}.pkl'.format(hidden_size, batches), 'wb') as fout:
    pickle.dump(my_decoder, fout)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1549628766161/work/torch/csrc/generic/serialization.cpp:23

In [ ]:
def translate(kopi_order):
    output_words = translator(my_encoder, my_decoder, variable_from_sent(kopi_order, english_vocab))
    print(output_words)
    output_sentence = [indo_vocab[i] for i in output_words[1:output_words.index(1)]]
    return ' '.join(output_sentence)

In [ ]:
translate('French Muslims fined for face veils')

In [ ]:
translate('child nutrition')